# Introduction

This notebook walks through the full analysis pipeline for exploring career patterns and predicting Cambridge attendance among data professionals. It is organized into sections with narrative explanations and code cells you can run interactively.

Sections:

1. Setup: imports and configuration

2. Data 

3. Feature engineering (including fuzzy skill grouping)

4. Exploratory skill analysis (Chi-square, plots)

5. Model pipelines (Logistic, RF, ET, XGBoost)


## 1. Setup

In [1]:
import os
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rapidfuzz import process, fuzz
import ast
from datetime import datetime


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

## 2. Data 

### 2.1 Data loading

First, we load the raw anonymised admissions dataset exported from the LinkedIn scraping pipeline. This single file contains all profiles, universities, skills, and experiences.

In [2]:
df = pd.read_csv(r"../datasets/anonymised admissions dataset")

# Expect ~656 entries with columns: school_1, school_2, skills, experiences, etc.

### 2.2 Preprocessing: separate Cambridge vs non-Cambridge profiles

Creating a column to capture whether they have studied at the University of Cambridge.

Creating a column to record other universities attended.

To do this, we start by filtering out for only records where either school_1 or school_2 contain "Cambridge"

In [3]:
# Remove entries missing both school_1 and school_2
df1 = df.dropna(subset=['school_1','school_2']).copy()

# Identify Cambridge profiles
df2 = df1[df1['school_1'].str.contains('Cambridge', case=False, na=False) |
          df1['school_2'].str.contains('Cambridge', case=False, na=False)].copy()

df2['Cambridge?'] = 1

df2['non-Cambridge uni'] = np.where(
    df2['school_1'].str.contains('Cambridge', case=False, na=False),
    df2['school_2'], df2['school_1'])

### 2.3 Building two datasets

We now repeat the process for records that do not contain "Cambridge."

Since both school records include two different university fields, we have two approaches for creating the dataset:

1. Duplication Method (dup_df): Data from two schools per profile was split into separate rows, increasing diversity but introducing some synthetic noise.

2. Arbitrary Method (arb_df): Only one school (typically the most recent) was selected, ensuring greater reliability at the expense of some nuance.

For this project, both datasets were generated, and similar models were run on each to better explore the nuances within the data.

In [4]:
# Profiles without Cambridge
df3 = df1.loc[~df1.index.isin(df2.index)].copy()

# Duplication method: split each row into two by school_1 and school_2
rows = []
for _, row in df3.iterrows():
    for school in [row['school_1'], row['school_2']]:
        new = row.copy()
        new['school_1'] = school
        new['non-Cambridge uni'] = school
        rows.append(new)
dup_df = pd.DataFrame(rows)
dup_df['Cambridge?'] = 0

dup_df = pd.concat([dup_df, df2], ignore_index=True)


# Arbitrary method: one row per profile (take school_1)
arb_df = df3.copy()
arb_df['non-Cambridge uni'] = arb_df['school_1']
arb_df['Cambridge?'] = 0

arb_df = pd.concat([arb_df, df2], ignore_index=True)

These two DataFrames, dup_df and arb_df, are used downstream for feature engineering and modeling.

## 4. Feature Engineering

### 4.1 Classifying based on university prestige

I wanted to investigate the importance of going to a Russel group university affects admissions to Cambridge.

To reflect the difference in prestige, I have grouped the universities of Oxford and Cambridge differently to other Russel group universities.

In [5]:
Oxbridge =["University of Oxford","University of Cambridge"]
Russel = [
    "University of Birmingham",
    "University of Bristol",
    "Cardiff University",
    "Durham University",
    "University of Edinburgh",
    "University of Exeter",
    "University of Glasgow",
    "Imperial College London",
    "King's College London",
    "University of Leeds",
    "University of Liverpool",
    "The London School of Economics and Political Science (LSE)",
    "University of Manchester",
    "Newcastle University",
    "University of Nottingham",
    "University of Sheffield",
    "University of Southampton",
    "UCL",
    "University of Warwick",
    "University of York",
    "Queen Mary University of London",
    "Queen's University Belfast"
]


In [6]:
def classify_uni(uni):
    if uni in Russel:
        return 1
    elif uni in Oxbridge:
        return 2
    else:
        return 0
    
dup_df["Uni Class"] = dup_df["non-Cambridge uni"].apply(classify_uni)
arb_df["Uni Class"]=arb_df["non-Cambridge uni"].apply(classify_uni)


### 4.2 Including relevant experiences in leadership and data

During data exploratory analysis, I found that data in the "experiences" feature is a list of dictionaries currently being stored as a string. To extract meaningful information, I have created fields capturing the months of experience in data related fields and leadership positions.

In [7]:
dup_df['experiences'] = dup_df['experiences'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
arb_df['experiences']=arb_df['experiences'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)


In [8]:
def calculate_months(start, end):
    """Calculate the difference in months between two dates."""
    if end is None:
        end = datetime.today().strftime('%Y-%m-%d') 
    start_date = datetime.strptime(start, '%Y-%m-%d')
    end_date = datetime.strptime(end, '%Y-%m-%d')
    
    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)


def extract_experience(experiences):
    data_months = 0
    leader_months = 0

    if isinstance(experiences, list):
        for exp in experiences:
            start = exp.get('start_date')
            end = exp.get('end_date')
            title = exp.get('title', '').lower()

            if not start:
                continue

            months = calculate_months(start, end)

            if any(word in title for word in ['data', 'analytics', 'ml', 'ai', 'scientist', 'engineer']):
                data_months += months

            if any(word in title for word in ['president', 'vice president', 'founder', 'head', 'CEO', 'COO']):
                leader_months += months

    return data_months,leader_months


In [9]:
dup_df[['data_months', 'leader_months']] = dup_df['experiences'].apply(lambda x: pd.Series(extract_experience(x)))

arb_df[['data_months', 'leader_months']] = arb_df['experiences'].apply(lambda x: pd.Series(extract_experience(x)))

### 4.3 Exploring the effect of QS Rankings and handling null values

In [10]:
qs=pd.read_csv(r"../datasets/qs-world-rankings-2025")

In [11]:
dup_df = dup_df.merge(qs[['Institution Name', '2025 Rank']], 
                         how='left', 
                         left_on='non-Cambridge uni', 
                         right_on='Institution Name')

arb_df = arb_df.merge(qs[['Institution Name', '2025 Rank']], 
                         how='left', 
                         left_on='non-Cambridge uni', 
                         right_on='Institution Name')

In this process, it was found that a number of records were falling out due to small differences in the university names. To handle these, I considered using a fuzzy match but chose against it since some of the values in "non-Cambridge uni" are that of sixth form colleges. To avoid incorrectly matching these, I chose to handle most fallouts by hand.

While this was time inefficient, it gave me greater control over the quality of my data.

In [12]:
unmatched = dup_df[dup_df['2025 Rank'].isna()]['non-Cambridge uni'].unique()

In [13]:
rank_overrides = {
    "University of Warwick": 69,
    "University of Warwick - Warwick Business School": 69,
    "Massachusetts Institute of Technology": 1,
    "University of Sydney": 18,
    "Indian Institute of Technology, Madras": 227,
    "Trinity College Dublin": 87,
    "Cardiff University / Prifysgol Caerdydd": 186,
    "City St George’s, University of London": 352,
    "Københavns Universitet - University of Copenhagen": 100,
    "University of Exeter": 169,
    "University of Naples ‘Parthenope’": 1100,
    "Saïd Business School, University of Oxford": 3,
    "University of Canterbury": 261,
    "UC Santa Barbara": 178,
    "Middlesex University (Hornsey College of Art)": 725,
    "University of Hamburg": 191,
    "Royal Holloway, University of London": 477,
    "University of Benin": 1350,
    "Vellore Institute of Technology": 795,
    "Imperial College Business School": 2,
    "Universidad Politécnica de Madrid": 321,
    "Kingston University": 605,
    "Univeristy of Cambridge": 5,
    "Delhi University": 328
}


def apply_rank_overrides(df):
    df['2025 Rank'] = df.apply(
        lambda row: rank_overrides.get(row['non-Cambridge uni'], row['2025 Rank']),
        axis=1
    )
    return df


dup_df = apply_rank_overrides(dup_df)
arb_df = apply_rank_overrides(arb_df)


As my dataset is small, I chose to handle all other null values differently.

On investigating the QS ranking dataset, I noticed that the dataset only included the top 1500 universities.

Furthermore, I observed that after the first 600 universities the rankings are a range. To handle this, I decided to assign such records the average of the range.

In [14]:
dup_df["2025 Rank"]=dup_df["2025 Rank"].fillna(9999)
arb_df["2025 Rank"]=arb_df["2025 Rank"].fillna(9999)

In [15]:
dup_df["2025 Rank"]=dup_df["2025 Rank"].astype(str)
arb_df["2025 Rank"]=arb_df["2025 Rank"].astype(str)

def convert_ranges(value):
    if "-" in value:
        lower, upper = map(int, value.split('-'))
        return (lower + upper) // 2  
    else:
        return int(value)



dup_df['2025 Rank'] = dup_df['2025 Rank'].apply(convert_ranges)
arb_df['2025 Rank'] = arb_df['2025 Rank'].apply(convert_ranges)

### 4.4 Fuzzy skill grouping and one-hot encoding selected skills

In [16]:
# Combine all unique skills from one dataset (repeat for both if needed)
raw_skills = pd.concat([dup_df['top_skill_1'], dup_df['top_skill_2'], dup_df['top_skill_3']]).dropna().unique().tolist()

# Build mapping via rapidfuzz
skill_map = {}
for s in raw_skills:
    match, score, _ = process.extractOne(s, raw_skills, scorer=fuzz.WRatio)
    skill_map[s] = match if score >= 80 else s

# Apply mapping to create fuzzy columns
def apply_fuzzy(df):
    for col in ['top_skill_1','top_skill_2','top_skill_3']:
        df[col + '_fuzzy'] = df[col].map(skill_map).fillna(df[col])
    return df

dup_df = apply_fuzzy(dup_df)
arb_df = apply_fuzzy(arb_df)

In [17]:
SIG_SKILLS = ["SQL","Microsoft Excel","Engineering","Mathematical Modeling",
              "Matlab","Communication","Data Science","Physics"]
for df in [dup_df, arb_df]:
    df['all_skills'] = df[['top_skill_1_fuzzy','top_skill_2_fuzzy','top_skill_3_fuzzy']].values.tolist()
    for s in SIG_SKILLS:
        col_name = f"skill_{s.replace(' ', '_')}"
        df[col_name] = df['all_skills'].apply(lambda lst: int(s in lst))
    df.drop(columns=['all_skills'], inplace=True)



## 5. Model Pipelines

We start by picking relevant features and defining a helper to prepare X,y.

In [18]:
BASE = ["Uni Class","data_months","leader_months","2025 Rank"]
SKILL_COLS = [f"skill_{s.replace(' ','_')}" for s in SIG_SKILLS]
FEATURES = BASE + SKILL_COLS

def prep(df):
    X = df[FEATURES].astype(float)
    y = df['Cambridge?']
    return X,y

We will now define our models and parameter grids.

In [19]:
MODELS = {
    'Logistic': (LogisticRegression(max_iter=1000, random_state=42), {'C':[0.1,1,10]}),
    'RandomForest': (RandomForestClassifier(random_state=42), {'n_estimators':[100,200],'max_depth':[None,10]}),
    'ExtraTrees': (ExtraTreesClassifier(random_state=42), {'n_estimators':[100,200],'max_depth':[None,10]}),
    'XGBoost': (XGBClassifier(use_label_encoder=False,eval_metric='logloss',random_state=42), {'n_estimators':[100,200],'max_depth':[None,10]})
}


We will now train and evaluate on all models on both datasets.

In [20]:
OUTPUT_DIR = 'results'
os.makedirs(OUTPUT_DIR, exist_ok=True)


def run_all(df, ds_name):
    X, y = prep(df)
    ds_folder = os.path.join(OUTPUT_DIR, ds_name)
    os.makedirs(ds_folder, exist_ok=True)
    for model_name, (clf, grid) in MODELS.items():
        pipe = Pipeline([('smote', SMOTE(random_state=42)), ('clf', clf)])
        grid_params = {f'clf__{k}': v for k, v in grid.items()}
        
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
        gs = GridSearchCV(pipe, grid_params, cv=5, scoring='f1', n_jobs=-1)
        gs.fit(Xtr, ytr)
        preds = gs.predict(Xte)
        out_path = os.path.join(ds_folder, f"{model_name}.txt")
        with open(out_path, 'w') as f:
            f.write(f"Dataset: {ds_name}\n")
            f.write(f"Model: {model_name}\n")
            f.write(f"Best Params: {gs.best_params_}\n")
            f.write(f"Accuracy: {accuracy_score(yte, preds):.4f}\n")
            f.write(f"ROC-AUC: {roc_auc_score(yte, preds):.4f}\n")
            f.write("Classification Report:\n")
            f.write(classification_report(yte, preds))
            f.write("\nConfusion Matrix:\n")
            f.write(str(confusion_matrix(yte, preds)))



def run_all_models():
    run_all(dup_df, 'duplication')
    run_all(arb_df, 'arbitrary')


run_all_models()

C:\Users\anagh\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\anagh\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [21]:
for ds_name, df_ in [('dup_df', dup_df), ('arb_df', arb_df)]:
    X_full, y_full = prep(df_)
    clf = MODELS['XGBoost'][0]
    pipe = Pipeline([('smote', SMOTE(random_state=42)), ('clf', clf)])
    pipe.fit(X_full, y_full)

    explainer = shap.TreeExplainer(pipe.named_steps['clf'])
    shap_values = explainer.shap_values(X_full)

    shap_dir = os.path.join('results', 'shap', ds_name)
    os.makedirs(shap_dir, exist_ok=True)

    fig = plt.figure()
    shap.summary_plot(shap_values, X_full, feature_names=FEATURES, show=False)
    fig.savefig(os.path.join(shap_dir, 'xgboost_summary.png'), bbox_inches='tight')
    plt.close(fig)

C:\Users\anagh\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\anagh\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


To conclude, here are the **three key insights** from my analysis:

1. **University Prestige is the Strongest Predictor**  
   Prior attendance at an Oxbridge or Russell Group institution (captured by `Uni Class` and QS `2025 Rank`) consistently had the highest feature importance and SHAP values, underscoring the role of institutional pedigree in Cambridge admissions predictions.

2. **Technical Skills Provide Nuanced Signals**  
   Of the validated skills, **SQL**, **Microsoft Excel**, and **Physics** emerged as meaningful predictors—sometimes correlating negatively—highlighting that even strong technical proficiencies may not uniformly increase admission probability and may interact with other factors.

3. **Leadership Experience Reflects Career Stage Bias**  
   Surprisingly, greater months in leadership roles tended to negatively influence predicted admission likelihood. A plausible interpretation is that candidates further along in their careers (with longer leadership tenures) are less likely to pursue full-time postgraduate study, illuminating a career trajectory effect.
